# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [8]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [23]:
OLLAMA_API="http://localhost:11434/api/chat"
HEADERS={"content-type":"application/json"}
MODEL="llama3.2"

In [24]:
system_prompt="You are an assistant who is excellent with summarizing content without losing the key idea. You are expected to generate a single subject line for the email."

In [25]:
user_prompt="""
Hi John, thanks again for all the work you’ve done as we near the beta \
launch of the VeggieVault recipe platform—I wanted to consolidate everything \
into one place before things get too chaotic. We’re still targeting July 5 for \
internal beta, and most stakeholder feedback has been addressed (around 80% complete), \
though a few remaining issues could impact demo polish: filters for “under 30 minutes” \
and “gluten-free” still don’t combine correctly, homepage hero image alt text is missing \
(flagged by accessibility), and the final count of imported recipes (goal is 10,000+) \
needs to be confirmed to fix the progress bar logic—can you make sure these are done \
by July 2 or let me know if we need to adjust the demo script? On the onboarding front, \
initial testers completed the recipe discovery tutorial successfully, but some UX issues \
popped up: people are confusing “bookmark recipe” with “add to pantry,” and it’s unclear \
whether saved pantry items are per-user or global (they are personalized)—maybe a tooltip \
would help? If design has time before July 1, it’d be great to fix it now; otherwise we’ll \
defer. I’ve also drafted a feedback survey for the ~25 internal testers and shared it in the \
Product Slack—please review it before EOD Friday. We should also plan a post-beta analytics \
and roadmap sync with Product, UX, and Eng—suggesting July 10 or 11—can you confirm your \
availability? Meanwhile, huge props to you and the data team for improving ingredient \
taxonomy; early SEO tests are encouraging. One note: on several recipe pages, “vegan” \
and “vegetarian” tags appear together even when dairy is present—let’s add a validation \
check for conflicting tags before recipes are published. Also, a few imported recipes \
(like from BBC GoodFood) are missing images—can we define a fallback strategy? I’ll \
update the project board tonight with open tasks and dependencies. Let’s stay close \
on Slack this week—appreciate everything, we’re almost there!
"""

In [30]:
messages = [
    {"role":"user", "content":user_prompt}
]

In [31]:
payload = {
    "model":MODEL,
    "messages":messages,
    "stream": False
}

In [32]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [33]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

This is an email to John, addressing several issues and tasks related to the VeggieVault recipe platform's beta launch. Here are some key points:

1. The deadline for internal beta is still July 5, but a few issues need to be addressed: 
   - Filters for "under 30 minutes" and "gluten-free" don't combine correctly.
   - Homepage hero image alt text is missing (flagged by accessibility).
   - Final count of imported recipes needs to be confirmed.

2. For the onboarding, there are a couple of UX issues:
   - Users are confusing "bookmark recipe" with "add to pantry."
   - It's unclear whether saved pantry items are per-user or global.
   A tooltip could help clarify this.

3. The developer team has drafted a feedback survey for internal testers and shared it in the Product Slack channel.

4. They plan an analytics and roadmap sync for July 10 or 11, which will include representatives from Product, UX, and Eng.

5. There's also a suggestion to add a validation check for conflicting tags (

In [34]:
import ollama

In [37]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Here's a response to your email:

Hi [Name],

Thanks for the detailed summary of our progress leading up to the internal beta launch! I'm glad to hear that 80% of stakeholder feedback has been addressed, and I'll make sure to focus on the remaining issues by July 2.

Regarding the filters, I've taken a look at the code and believe we can fix the combination issue for "under 30 minutes" and "gluten-free". However, please confirm that this is acceptable, or we may need to adjust the demo script accordingly. For the homepage hero image alt text, I'll add it immediately to address accessibility concerns.

For the final count of imported recipes, I've checked on our progress, and we're currently at 8,500+ recipes. We should be able to reach the goal of 10,000 by July 5. If this changes, please let me know, and we'll adjust the progress bar logic accordingly.

On the onboarding front, I agree that a tooltip would help clarify the difference between "bookmark recipe" and "add to pantry". I'll